# Summarization Task

There are 2 parts in this exercise.

#### Part-1 

1. Start by exploring models on the Hugging Face Hub. Identify 2 models, one for each of the following 
   * Extractive summarization  (*Hint:* facebook/bart...)
   * Abstractive summarization (*Hint:* Falconsai/text_...)
2. Use the *Inference client class* to summarize the text with both models
   * Compare the results
   * Which model worked out better for you?
3. Measure the inference time for the better model
   * Place the **client.summarize(text)** call in its own cell, with **%%time** on top of cell
  
#### Part-2

4. Use the better model with Pipeline class to summarize the text
   * Place the inference invocation call in its own cell i.e., **summarization_pipeline(text)**
   * Use **%%time** to measure the time it took to summarize the document 
5. Which mechanism worked out better from performance perspective?
   * Pipeline?
   * Inference Cli
ent i.e., Endpoint?

**(Optional)*in the Q&A * Sha
 your Sample text for summarization##"""ers in the Q&A :)

rizFueling your body with a healthy diet isn't just about looking good, it's about building a vibrant
foundation for a life well-lived. Imagine a symphony of vibrant vegetables, a chorus of juicy fruits, and a 
rhythm of whole grains and lean protein – that's the melody of a healthy diet, 
nourishing your cells and empowering your spirit. Ditch the processed sirens and sugary temptations, 
and embrace the natural orchestra of flavors and textures that nature offers. Let leafy greens be 
your verdant bassline, their antioxidants dancing on your tongue. Citrus fruits, like sunshine-
kissed trumpets, add a bright, refreshing melody. Whole grains, the sturdy timpani, provide 
sustained energy, while lean protein, a soulful cello, builds and repairs your body's instruments. 
Don't forget the playful percussion of nuts and seeds, bursting with essential nutrients, and the 
creamy, comforting oboe of legumes, rich in fiber and protein. This symphony of flavors isn't just 
delicious, it's a powerful conductor, harmonizing your gut, boosting your immunity, and keeping your 
energy levels soaring. So, ditch the culinary cacophony of processed foods, and embrace the healthy 
concerto of a vibrant, colorful diet. Let your body be the maestro, conducting a symphony of well-
being, one delicious bite at a time.ry of its construction."""


**Note:**
If you are running on Google Colab:

1. Install the packages in runtime
2. Restart the kernel before running the cells

**Additional libraries**
* Some Models require addtional libraries to be installed e.g., google/pegasus-large has a dependency on sentencepiece package !!
* If you come across a model like that, either install dependency or try another model

#### Google Colab
If you are running the code in Google colab, install the packages by uncommenting/running the cell below

In [15]:
# !pip install transformers torch huggingface_hub
# ! uv add huggingface_hub torch transformers

In [1]:
from dotenv import load_dotenv
import os

# Abstractive & Extractive summarization
MODEL_EXTRACTIVE  = 'facebook/bart-large-cnn'

MODEL_ABSTRACTIVE = 'Falconsai/text_summarization'

######## Alternatives ######

# Another example of abstractive summarization model
# MODEL_ABSTRACTIVE = 'google/flan-t5-small'

# You need to have sentencepiece installed to convert a slow tokenizer to a fast one.
# MODEL_ABSTRACTIVE = 'google/pegasus-large'

############################



text = """Fueling your body with a healthy diet isn't just about looking good, it's about building a vibrant
foundation for a life well-lived. Imagine a symphony of vibrant vegetables, a chorus of juicy fruits, and a 
rhythm of whole grains and lean protein – that's the melody of a healthy diet, 
nourishing your cells and empowering your spirit. Ditch the processed sirens and sugary temptations, 
and embrace the natural orchestra of flavors and textures that nature offers. Let leafy greens be 
your verdant bassline, their antioxidants dancing on your tongue. Citrus fruits, like sunshine-
kissed trumpets, add a bright, refreshing melody. Whole grains, the sturdy timpani, provide 
sustained energy, while lean protein, a soulful cello, builds and repairs your body's instruments. 
Don't forget the playful percussion of nuts and seeds, bursting with essential nutrients, and the 
creamy, comforting oboe of legumes, rich in fiber and protein. This symphony of flavors isn't just 
delicious, it's a powerful conductor, harmonizing your gut, boosting your immunity, and keeping your 
energy levels soaring. So, ditch the culinary cacophony of processed foods, and embrace the healthy 
concerto of a vibrant, colorful diet. Let your body be the maestro, conducting a symphony of well-
being, one delicious bite at a time."""


load_dotenv('E:\\Code\\gen-ai-app-dev-course\\.env')
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

## Part-1

### HuggingFace Inference Endpoint API
This will use the model hosting on the Hugging Face portal

https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.

Class

https://huggingface.co/docs/huggingface_hub/v0.20.2/en/package_reference/inference_client#huggingface_hub.InferenceClient

Supported tasks

https://huggingface.co/docs/huggingface_hub/guides/inference#supported-tasks

NOTE:

Sometimes API calls fail due to heavy usage of the model on HF. If you get a invocation error, try a again/few times.

#### Use Extractive Model

In [2]:
from huggingface_hub import InferenceClient

# Create the Inference Client
client = InferenceClient(model=MODEL_EXTRACTIVE, token=HUGGINGFACEHUB_API_TOKEN)

In [3]:
%%time

# Summarize using the summarize function on the client class
client.summarization(text)

CPU times: total: 62.5 ms
Wall time: 5.02 s


SummarizationOutput(summary_text="Ditch the processed sirens and sugary temptations and embrace the natural orchestra of flavors and textures that nature offers. Let leafy greens be your bassline, their antioxidants dancing on your tongue. Citrus fruits, like sunshine-kissed trumpets, add a bright, refreshing melody. Whole grains provide sturdy timpani, while lean protein, a soulful cello, builds and repairs your body's instruments.")

#### Use Abstractive Model

In [4]:
# Create the Inference Client
client = InferenceClient(model=MODEL_ABSTRACTIVE, token=HUGGINGFACEHUB_API_TOKEN)

In [10]:
%%time

# Summarize using the summarize function on the client class
client.summarization(text)

CPU times: total: 0 ns
Wall time: 4.03 s


SummarizationOutput(summary_text="Imagine a symphony of vibrant vegetables, a chorus of juicy fruits, and a rhythm of whole grains and lean protein – that's the melody of a healthy diet . Let leafy greens be your verdant bassline, their antioxidants dancing on your tongue .")

## Part-2

### Use HuggingFace Pipeline API (MODEL_ABSTRACTIVE)

**Note:**
* You MUST have disk space for the model artefacts, otherwise pipeline will fail !!
* The pipeline downloads the model weights, loads them in memory to run the inference. 

In [7]:
from transformers import pipeline

# Specify the task
TASK = "summarization"

summarization_pipeline = pipeline(TASK,model = MODEL_ABSTRACTIVE, max_new_tokens=100)

Device set to use cpu


In [9]:
%%time

summarization_pipeline(text)[0]['summary_text']

CPU times: total: 6.09 s
Wall time: 1.55 s


"Imagine a symphony of vibrant vegetables, a chorus of juicy fruits, and a rhythm of whole grains and lean protein – that's the melody of a healthy diet . Let leafy greens be your verdant bassline, their antioxidants dancing on your tongue ."